### Importamos las librerías necesarias

In [187]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Iniciamos con la lectura de los datos

In [188]:
df = pd.read_excel("real_estate_data.xlsx", index_col = [0])

#### Formateo de los nombres de las columnas para remover espacios

In [189]:
def format_cols_spaces(columns):
  return {key:key.replace(" ","_") for key in columns}

### Funcion para estandarizar la data

In [190]:
def standardize_data(data):
    new_data = data.copy()
    for col in new_data.columns:
        new_data[col] = (new_data[col] - new_data[col].mean()) / new_data[col].std()
    return new_data

In [191]:
columns_replace = format_cols_spaces(df.columns)
df = df.rename(columns = columns_replace)
df_train = df.loc[0:315].copy()
df_test = df.loc[316:].copy()
df_train_norm = standardize_data(df_train)
df_test_norm = standardize_data(df_test)

### Creamos la clase NeuralNetwork

In [192]:
class NeuralNetwork:
  def __init__(self, activation, eps, learning_rate, epochs):

    #Inicializacion de pesos
    self.W1 = np.random.random((5,6))
    self.b1 = np.random.random((5,1))

    self.W2 = np.random.random((1,5))
    self.b2 = np.random.random((1,1))

    #Inicializacion de hiperparametros
    self.eps = eps
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.activation = self.sigmoid if activation == "sigmoid" else self.relu

  def relu(self,X):
    return np.maximum(0,X)
  
  def sigmoid(self,X):
    return 1/(1+np.exp(-X))

  def forward(self, X):
    zi = self.W1@X + self.b1
    zi = self.activation(zi)
    y_hat = self.W2@zi + self.b2
    return y_hat 
  
  def funcion_objetivo(self, X, y):
    return 0.5*np.mean((self.forward(X).T - y)**2)
  
  def numerical_gradient(self, X, y):
      # Calcular el gradiente numérico
      eps = self.eps
      grads = {}
      for param_name in ["W1", "b1", "W2", "b2"]:
          param = getattr(self, param_name)
          loss_mat = np.zeros_like(param)
          ## For loop para W1
          for i in range(param.shape[0]):
            for j in range(param.shape[1]):
                original_value = param[i, j]
                param[i, j] = original_value + eps
                grad_plus = self.funcion_objetivo(X, y)
                param[i, j] = original_value - eps
                grad_minus = self.funcion_objetivo(X, y)
                param[i, j] = original_value
                loss_mat[i, j] = (grad_plus - grad_minus) / (2 * eps)
          grads[param_name] = loss_mat
      return grads
  
#funcion fit y loop de entrenamiento
  def fit(self, X, y):
    learning_rate = self.learning_rate
    epochs = self.epochs
    print("Entrenando...")
    # Inicializar historial de pérdidas
    loss_history = []

    # Ciclo de entrenamiento
    for epoch in range(epochs):
        # Calculamos los gradientes
        grads = self.numerical_gradient(X, y)

        # Actualizamos los pesos y bias
        for param_name in ["W1", "b1", "W2", "b2"]:
            param = getattr(self, param_name)
            grad = grads[param_name]
            param -= learning_rate * grad
        loss = self.funcion_objetivo(X, y)
        loss_history.append(loss)
    return loss_history[-1]



In [194]:
df_train_X = df_train.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y = df_train[['Y_house_price_of_unit_area']]

df_train_X_norm = df_train_norm.drop(['Y_house_price_of_unit_area'], axis = 1)
df_train_y_norm = df_train_norm[['Y_house_price_of_unit_area']]

In [195]:
train_X_values = df_train_X.values.T
train_y_values = df_train_y.values

train_X_norm_values = df_train_X_norm.values.T
# train_y_norm_values = df_train_y_norm.values


In [196]:
train_y_values.shape

(315, 1)

In [193]:
for eps in range(2,5):
    for learning_rate in range(2,5):
        for epochs in range(2,5):
            for activation in ["sigmoid", "relu"]:
                nn = NeuralNetwork(activation, 10**(-eps), 10**(-learning_rate), 10**(epochs))
                nn.fit(train_X_values, train_y_values)

In [110]:
df_test_X = df_test.drop(['Y_house_price_of_unit_area'], axis = 1)
df_test_y = df_test[['Y_house_price_of_unit_area']]

df_test_X_norm = df_test_norm.drop(['Y_house_price_of_unit_area'], axis = 1)
df_test_y_norm = df_test_norm[['Y_house_price_of_unit_area']]

In [148]:
preds = neural_1.forward(df_test_X_norm.values.T)


In [150]:
preds.shape

(1, 99)

In [151]:
mse = np.mean((preds.T - df_test_y.values)**2)
mse

66.00471078521562